In [23]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import statsmodels.formula.api as smf
# from linearmodels.panel import PanelOLS

In [28]:
wf = pd.read_csv("data/wf.csv")
city_yb = pd.read_csv("data/city_yb.csv")

## 2020 treatment vs control

In [106]:
wf["temp2"] = wf["temp"] ** 2
wf["l_aqi"] = np.log(1 + wf["aqi"])
wf["l_pm"] = np.log(1 + wf["pm"])
wf2020 = wf[(wf["daynum"] >= 8401) & (wf["daynum"]<= 8461)].dropna()

In [112]:
# subtract entity means to create time and city invariant variables
out_vars = ["aqi", "l_aqi", "pm", "l_pm"]
weather_vars = ["prec", "snow", "temp", "temp2"]
wf2020_city_mean = wf2020.groupby(["city_code"])[out_vars + weather_vars].transform(np.mean)
wf2020_day_mean = wf2020.groupby(["daynum"])[out_vars + weather_vars].transform(np.mean)

wf2020_inv = wf2020.copy()
wf2020_inv[out_vars + weather_vars] = wf2020_inv[out_vars + weather_vars] - wf2020_city_mean
wf2020_inv[out_vars+ weather_vars] = wf2020_inv[out_vars+ weather_vars] - wf2020_day_mean
wf2020_inv

,Unnamed: 0,date,prec,snow,temp,aqi,co,no2,o3,pm10,...,hubei,day,month,year,daynum,north,city_code,temp2,l_aqi,l_pm
365,366,20200101,-9.362837,-74.888772,-5.116662,-110.040788,0.671667,44.291668,13.708333,47.125000,...,0,1,1,2020,8401,1,54613,-24.937805,-4.543772,-4.160675
366,367,20200102,-8.388243,-69.284371,-4.404594,-125.457453,1.002500,54.833332,10.500000,68.208336,...,0,2,1,2020,8402,1,54613,-51.970291,-4.574684,-4.170973
367,368,20200103,-11.590199,-72.505010,-3.374476,-150.521765,1.220000,64.250000,15.958333,77.333336,...,0,3,1,2020,8403,1,54613,-73.654079,-4.754514,-4.289581
368,369,20200104,-21.291443,-64.669849,-5.171529,-176.196585,1.222500,62.500000,24.291666,72.125000,...,0,4,1,2020,8404,1,54613,-71.311025,-4.966757,-4.432369
369,370,20200105,-17.611484,-50.394329,-2.992178,-117.866421,1.159583,59.916668,12.250000,69.833336,...,0,5,1,2020,8405,1,54613,-83.217484,-4.370740,-4.397247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136950,136951,20200226,-13.005500,-52.127001,-2.305948,-76.495863,0.545000,19.625000,63.416668,146.958328,...,0,26,2,2020,8457,1,41982,-127.084915,-4.165209,-3.812628
136951,136952,20200227,-43.702427,-54.313260,-0.693159,-54.299305,0.643750,26.625000,51.833332,120.375000,...,0,27,2,2020,8458,1,41982,-90.986511,-3.873437,-3.598150
136952,136953,20200228,-24.079100,-54.606251,1.688433,-87.196496,0.686667,32.777779,59.777779,116.388885,...,0,28,2,2020,8459,1,41982,-54.710885,-4.259463,-3.592303
136953,136954,20200229,-32.514441,-31.102905,1.139426,-68.378567,0.536250,29.500000,51.833332,98.666664,...,0,29,2,2020,8460,1,41982,-60.796033,-4.006151,-4.122448


In [114]:
# NOTE: std errs shld be clustered
X_1 = wf2020[["treat"]] # city fixed effects + date fixed effects
X_2 = wf2020[["treat"] + weather_vars] # + weather control
fits = []
for y_col in out_vars:
    # model only with lockdown + city and time fixed effects
    fits.append(LinearRegression().fit(X_1, wf2020_inv[y_col]))
    # print(sm.OLS(wf2020_inv[y_col], sm.add_constant(X_1)).fit().params)
    # model also with weather controls
    fits.append(LinearRegression().fit(X_2, wf2020_inv[y_col]))
    # print(sm.OLS(wf2020_inv[y_col], sm.add_constant(X_2)).fit().params)

for i in range(len(out_vars)):
    print(out_vars[i])
    print("no weather: ", fits[i*2].coef_)
    print("weather: ", fits[i*2+1].coef_)

aqi
no weather:  [-0.29883315]
weather:  [-0.97841528 -0.05064458  0.0074296  -0.04217605  0.02893243]
l_aqi
no weather:  [0.00832913]
weather:  [-0.00483157 -0.00092045  0.00011813  0.00074289  0.00033933]
pm
no weather:  [-0.41285745]
weather:  [-0.84579548 -0.02131683 -0.00200736 -0.00619765  0.01599116]
l_pm
no weather:  [0.00961903]
weather:  [-3.14837998e-03 -5.12737405e-04  1.60058599e-05  1.39845537e-03
  2.58380161e-04]


## 2020 vs 2019 control group Spring Festival